In [31]:
#importation
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from itertools import islice
import sys
import os

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
#loading data and displaying columns
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
#data_path = os.path.join(parent_dir, "data", "Week1_challenge_data_source.csv")
data_path2 = os.path.join(parent_dir, "data", "user_engage.csv")
data_path3 =  os.path.join(parent_dir, "data", "user_exp.csv")
models_dir = os.path.join(parent_dir, "models")
user_engagement = pd.read_csv(data_path2)
user_exp = pd.read_csv(data_path3)
user_engagement.drop('Unnamed: 0', inplace = True, axis =1)
user_engagement.head()

,Customer Number,Session Frequency,Session Duration,Total Data Volume,Google(UL/DL),Email(UL/DL),Gaming(UL/DL),Youtube(UL/DL),Netflix(UL/DL),Social(UL/DL)
0,3.360100e+10,1,116720.0,8.786906e+08,2905912.0,3701304.0,292426453.0,18355943.0,17855187.0,1570185.0
1,3.360100e+10,1,181230.0,1.568596e+08,4414096.0,937385.0,609920783.0,39359124.0,35565545.0,1933278.0
2,3.360100e+10,1,134969.0,5.959665e+08,10229119.0,3363124.0,229980251.0,34425237.0,23751202.0,1726277.0
3,3.360101e+10,1,49878.0,4.223207e+08,11811761.0,2070983.0,810387875.0,36534765.0,15092588.0,657493.0
4,3.360101e+10,2,37104.0,1.457411e+09,7748843.0,2110349.0,531237049.0,34222253.0,17539799.0,912788.0


In [34]:
usr_eng = user_engagement[['Customer Number' ,'Session Frequency','Session Duration','Total Data Volume']]
column = ['Customer Number' ,'Session Frequency','Session Duration','Total Data Volume']
for c in column:
    usr_eng[c]=user_engagement[c]

usr_eng.head()

,Customer Number,Session Frequency,Session Duration,Total Data Volume
0,3.360100e+10,1,116720.0,8.786906e+08
1,3.360100e+10,1,181230.0,1.568596e+08
2,3.360100e+10,1,134969.0,5.959665e+08
3,3.360101e+10,1,49878.0,4.223207e+08
4,3.360101e+10,2,37104.0,1.457411e+09


In [35]:
to_scale = usr_eng[['Session Frequency','Session Duration','Total Data Volume']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(to_scale)
normalized = pd.DataFrame(scaled, columns=['Session Frequency','Session Duration','Total Data Volume'])

In [36]:
from sklearn.cluster import KMeans
KMean_clust= KMeans(n_clusters=3, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)

KMeans(max_iter=1000, n_clusters=3)

In [37]:
normalized['Cluster']= KMean_clust.labels_

metrics = usr_eng[['Session Frequency','Session Duration','Total Data Volume']]
metrics['Cluster']=KMean_clust.labels_
metrics

,Session Frequency,Session Duration,Total Data Volume,Cluster
0,1,116720.0,8.786906e+08,1
1,1,181230.0,1.568596e+08,1
2,1,134969.0,5.959665e+08,1
3,1,49878.0,4.223207e+08,1
4,2,37104.0,1.457411e+09,0
...,...,...,...,...
106851,1,8810.0,7.146416e+08,1
106852,1,140988.0,4.803073e+08,1
106853,1,877385.0,2.321240e+08,1
106854,1,253030.0,5.962878e+08,1


In [38]:
clus_mean = metrics.groupby('Cluster').mean()
clus_mean

,Session Frequency,Session Duration,Total Data Volume
Cluster,,,
0,2.176229,213614.376774,1.125463e+09
1,1.042238,106985.013467,4.955072e+08
2,4.248401,569441.309395,2.280790e+09


In [39]:
centroids = KMean_clust.cluster_centers_
centroids = min_max_scaler.inverse_transform(centroids)
print(centroids)
usr_eng['Engagement Score'] = np.linalg.norm(usr_eng[['Session Frequency', 'Session Duration','Total Data Volume']].sub(np.array(centroids[0])), axis=1)
usr_eng

[[2.17566568e+00 2.13502844e+05 1.12493766e+09]
 [1.04210825e+00 1.06976349e+05 4.95490323e+08]
 [4.24582924e+00 5.69044983e+05 2.27937840e+09]]


,Customer Number,Session Frequency,Session Duration,Total Data Volume,Engagement Score
0,3.360100e+10,1,116720.0,8.786906e+08,2.462471e+08
1,3.360100e+10,1,181230.0,1.568596e+08,9.680780e+08
2,3.360100e+10,1,134969.0,5.959665e+08,5.289712e+08
3,3.360101e+10,1,49878.0,4.223207e+08,7.026170e+08
4,3.360101e+10,2,37104.0,1.457411e+09,3.324733e+08
...,...,...,...,...,...
106851,3.379000e+10,1,8810.0,7.146416e+08,4.102961e+08
106852,3.379000e+10,1,140988.0,4.803073e+08,6.446303e+08
106853,3.197020e+12,1,877385.0,2.321240e+08,8.928139e+08
106854,3.370000e+14,1,253030.0,5.962878e+08,5.286498e+08


In [40]:
user_exp.head()

,Customer Number,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL),No. of Handset
0,3.360100e+10,76.0,46.000000,2.156957e+07,1
1,3.360100e+10,99.0,31.000000,2.156957e+07,1
2,3.360100e+10,97.0,127.458589,2.156957e+07,1
3,3.360101e+10,248.0,84.000000,7.607247e+05,1
4,3.360101e+10,56844.0,119.000000,3.094040e+07,1


In [41]:
to_scale = user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(to_scale)
normalized = pd.DataFrame(scaled, columns=['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)'])
KMean_clust= KMeans(n_clusters=3, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)
metric=user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']]
metric['Cluster']=KMean_clust.labels_
metric

,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL),Cluster
0,76.0,46.000000,2.156957e+07,0
1,99.0,31.000000,2.156957e+07,0
2,97.0,127.458589,2.156957e+07,0
3,248.0,84.000000,7.607247e+05,0
4,56844.0,119.000000,3.094040e+07,1
...,...,...,...,...
106851,10365.0,52.000000,2.180450e+05,0
106852,116.0,40.000000,2.156957e+07,0
106853,1.0,127.458589,2.156957e+07,0
106854,33.0,127.458589,2.156957e+07,0


In [42]:
clus_mean = metric.groupby('Cluster').mean()
clus_mean

,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL)
Cluster,,,
0,4808.987393,167.202770,2.144464e+07
1,67287.492227,183.489250,3.930920e+07
2,209388.412491,397.105749,2.413433e+08


In [43]:
usr_eng.columns.tolist()

['Customer Number',
 'Session Frequency',
 'Session Duration',
 'Total Data Volume',
 'Engagement Score']

In [44]:
centroids = KMean_clust.cluster_centers_
centroids = min_max_scaler.inverse_transform(centroids)

satis=user_exp
satis['Experience Score'] = np.linalg.norm( user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']].sub(np.array(centroids[0])), axis=1)

# retreive engagement scores
satis['Engagement Score']=usr_eng['Engagement Score'].values
col = satis.loc[: , "Engagement Score":"Experience Score"]
satis['Satisfaction'] = user_exp[["Engagement Score","Experience Score"]].mean(axis=1)
satis=satis[["Engagement Score","Experience Score",'Satisfaction']]
satis.nlargest(10,'Satisfaction')


,Engagement Score,Experience Score,Satisfaction
66860,1.080550e+09,8.332765e+09,4.706657e+09
34123,5.278939e+08,7.920416e+09,4.224155e+09
42366,1.047222e+09,7.353454e+09,4.200338e+09
98213,2.138003e+09,5.858545e+09,3.998274e+09
6437,7.721295e+09,1.129741e+08,3.917135e+09
92923,7.389842e+09,2.287624e+08,3.809302e+09
13180,7.374706e+09,2.407386e+08,3.807722e+09
100468,2.043219e+09,5.074241e+09,3.558730e+09
13526,6.846235e+09,2.154056e+08,3.530820e+09
76363,6.766175e+09,1.459088e+08,3.456042e+09


In [45]:
from sklearn.linear_model import LinearRegression


x = satis[['Engagement Score','Experience Score']]
y = satis['Satisfaction']
x, y = np.array(x), np.array(y)
model = LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('coefficient determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)
y_pred = model.predict(x)
print('predicted response:', y_pred, sep='\n')

coefficient determination: 1.0
intercept: -9.5367431640625e-07
slope: [0.5 0.5]
predicted response:
[1.23186181e+08 4.84101637e+08 2.64548220e+08 ... 4.46469598e+08
 2.64387548e+08 4.92954746e+08]


In [47]:
# Saving model to file
import pickle

filename = 'pridict_satisfaction_model.sav'
file_loc = os.path.join(models_dir,filename)
pickle.dump(model, open(file_loc, 'wb'))
print("MODEL SAVED SUCCESSFULLY.")

MODEL SAVED SUCCESSFULLY.


In [18]:
x = satis[['Engagement Score','Experience Score']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized = pd.DataFrame(x_scaled,columns=['Engagement Score','Experience Score'])
KMean_clust= KMeans(n_clusters=2, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)
satis['Cluster']=KMean_clust.labels_
satis

,Engagement Score,Experience Score,Satisfaction,Cluster
0,3.831987e+08,1.804762e+07,2.006232e+08,0
1,3.386322e+08,1.804762e+07,1.783399e+08,0
2,1.004746e+08,1.804762e+07,5.926112e+07,0
3,7.317119e+07,3.885641e+07,5.601380e+07,0
4,9.619191e+08,8.676673e+06,4.852979e+08,1
...,...,...,...,...
106851,2.191498e+08,3.939907e+07,1.292744e+08,0
106852,1.518457e+07,1.804762e+07,1.661610e+07,0
106853,2.633690e+08,1.804762e+07,1.407083e+08,0
106854,1.007961e+08,1.804762e+07,5.942184e+07,0


In [19]:
satis.groupby('Cluster')[['Engagement Score','Experience Score']].mean()

,Engagement Score,Experience Score
Cluster,,
0,2.434966e+08,2.904939e+07
1,1.261791e+09,5.280240e+07


In [20]:
satis.head()

,Engagement Score,Experience Score,Satisfaction,Cluster
0,3.831987e+08,1.804762e+07,2.006232e+08,0
1,3.386322e+08,1.804762e+07,1.783399e+08,0
2,1.004746e+08,1.804762e+07,5.926112e+07,0
3,7.317119e+07,3.885641e+07,5.601380e+07,0
4,9.619191e+08,8.676673e+06,4.852979e+08,1


In [26]:
import sqlite3
con = sqlite3.connect ('tellco.db')
cursor = con.cursor ()
try:
    cur.execute ('create table user_satisfaction_level (user_id, eng_score number, exp_score number, Satisfaction)')
except :
    print('Table exists.... continuing')


satis['user_id']=satis.index
satis[['user_id','Engagement Score','Experience Score','Satisfaction']].to_sql('user_satisfaction_level', con, if_exists='replace', index=False)

satis

Table exists.... continuing


,Engagement Score,Experience Score,Satisfaction,Cluster,user_id
0,3.831987e+08,1.804762e+07,2.006232e+08,0,0
1,3.386322e+08,1.804762e+07,1.783399e+08,0,1
2,1.004746e+08,1.804762e+07,5.926112e+07,0,2
3,7.317119e+07,3.885641e+07,5.601380e+07,0,3
4,9.619191e+08,8.676673e+06,4.852979e+08,1,4
...,...,...,...,...,...
106851,2.191498e+08,3.939907e+07,1.292744e+08,0,106851
106852,1.518457e+07,1.804762e+07,1.661610e+07,0,106852
106853,2.633690e+08,1.804762e+07,1.407083e+08,0,106853
106854,1.007961e+08,1.804762e+07,5.942184e+07,0,106854


In [29]:
#print the first ten rows of user_satisfaction
cursor.execute("SELECT * FROM user_satisfaction_level LIMIT 10")
rows = cursor.fetchall()
for row in rows:
    print(row)

(0, 383198708.97185826, 18047624.474042132, 200623166.7229502)
(1, 338632230.29287195, 18047624.388388194, 178339927.34063008)
(2, 100474621.74727322, 18047624.39528747, 59261123.071280345)
(3, 73171189.43049018, 38856405.21272769, 56013797.32160893)
(4, 961919081.3857481, 8676673.340848396, 485297877.3632983)
(5, 119725446.09947418, 17285182.61902174, 68505314.35924795)
(6, 159231202.28375828, 17937284.183386516, 88584243.2335724)
(7, 162831509.4522763, 38857191.49822081, 100844350.47524855)
(8, 494640476.59248585, 3522715.103862145, 249081595.848174)
(9, 236971896.5223168, 18805983.82556606, 127888940.17394143)
